In [1]:
class thermal_generator:
    
    def __init__(self,power_out_min, power_out_max, time_up_min, time_down_min,must_run, 
                 ramp_up, ramp_down, cost, A_D, A_U, time_horizon ):
        self.time_up_min = time_up_min
        self.time_down_min = time_down_min
        self.must_run = must_run
        self.ramp_up = ramp_up
        self.ramp_down = ramp_down
        self.cost = cost
        self.A_D = A_D
        self.A_U = A_U
        self.power_level = []
        for level in range(0,time_horizon):
            self.power_level.append([0] + [x for x in range(round(power_out_min), round(power_out_max + 1))])
    
    def print_all_info(self):
        print("Minimum time up : {}".format(self.time_up_min))
        print("Minimum time down : {}".format(self.time_down_min))
        print("Must run time : {}".format(self.must_run))
        print("Ramp up : {}".format(self.ramp_up))
        print("Ramp down : {}".format(self.ramp_down))
        print("Costs : {} ".format(self.cost))
        print("Start up cost : {}".format(self.A_U))
        print("Start down cost : {}".format(self.A_D))
        print("Power levels : {}".format(self.power_level))

In [2]:
class wind_generator:
    
    def __init__(self,power_levels):
        self.power_level = []
        for level in power_levels:
            self.power_level.append([0] + [x for x in range(round(level[0]), round(level[1] + 1))])
        self.time_up_min = 0
        self.time_down_min = 0
        self.must_run = 0
        self.ramp_up = 0
        self.ramp_down = 0
        self.cost = 0
        self.A_D = 0
        self.A_U = 0
        
    def print_all_info(self):
        print("power maximum output : {}".format(self.power_level))

In [3]:
class network:
    
    def __init__(self, time_period, demand, reserves, thermal_gen = [], renewable_gen = []):
        self.time_period = time_period
        self.demand = demand
        self.reserves = reserves
        self.thermal_gen = thermal_gen
        self.renewable_gen = renewable_gen
        self.gen = []
    
    def add_thermal_generator(self, thermal_generator):
        self.thermal_gen.append(thermal_generator)
    
    def add_renewable_generator(self, renew_generator):
        self.renewable_gen.append(renew_generator)
    
    def get_all_generators(self):
        self.gen = self.renewable_gen + self.thermal_gen
    
    def print_all_info(self):
        print("Time period : {}".format(self.time_period))
        print("Demand(Loads) over time : {}".format(self.demand))
        print("reserves over time : {}".format(self.reserves))
        print("Number of Thermal generators : {}".format(len(self.thermal_gen)))
        print("Number of renewable generators : {}".format(len(self.renewable_gen)))

In [4]:
def create_thermal_generator(json_object,time):
    return thermal_generator(json_object['power_output_minimum'],json_object['power_output_maximum'],
                             json_object['time_up_minimum'], json_object['time_down_minimum'],
                            json_object['must_run'],json_object['ramp_up_limit'],json_object['ramp_down_limit'],
                            json_object['piecewise_production'][0]['cost'],
                            json_object['startup'][0]['cost'],
                            0,time)

def create_renewable_generator(json_object):
    power_levels = []
    for x in range(0,len(json_object['power_output_minimum'])):
        power_levels.append((json_object['power_output_minimum'][x],json_object['power_output_maximum'][x]))
    return wind_generator(power_levels)

def create_network(json_object, time):
    net = network(json_object['time_periods'],json_object['demand'],json_object['reserves'])
    for x in json_object['thermal_generators']:
        gen = create_thermal_generator(json_object['thermal_generators'][x],time)
        net.add_thermal_generator(gen)
    for x in json_object['renewable_generators']:
        gen = create_renewable_generator(json_object['renewable_generators'][x])
        net.add_renewable_generator(gen)
    return net

In [5]:
# Import dependencies
import numpy as np
import os
import time
import math
import requests
import json
import datetime  

from azure.quantum import Workspace
from azure.quantum.optimization import Problem, ProblemType,SlcTerm, Term, HardwarePlatform, Solver
from azure.quantum.optimization import SimulatedAnnealing, ParallelTempering, HardwarePlatform, Tabu, SubstochasticMonteCarlo, PopulationAnnealing
from typing import List

# Connect to your Azure Quantum Workspace
workspace = Workspace (
    subscription_id = "<Subscription-id>",  # Add your subscription_id
    resource_group = "<resource_group>",   # Add your resource_group
    name = "<workspace name>",             # Add your workspace name
    location = "<location>"          # Add your workspace location (for example, "westus")
)

# Open the browser and sign in with the generated code (on first login)
workspace.login()

c:\users\mhd-a\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated method login. (This method is deprecated and no longer necessary to be called) -- Deprecated since version 0.17.2105.


In [6]:
def map_variable(i,k,t, K, T):
    var_name = "1{}{}{}".format(i,k,t)
    return int(var_name)

In [7]:
def UCproblem(net:network) -> Problem:
    terms = []
    ##### Cost function
    for t in range(0, net.time_period):
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                #terms.append(Term(c = A, indices = [map_variable(i,k,t,PowerMatrix.shape[0],len(l))]))
                terms.append(Term(c = round(net.gen[i].cost * net.gen[i].power_level[t][k]), indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)] ))
                #terms.append(Term(c = int(C * PowerMatrix.item((k,i)) * PowerMatrix.item((k,i))), indices = [map_variable(i,k,t,PowerMatrix.shape[0],len(l))] ))
                if t == 0:
                    terms.append(Term(c = round(net.gen[i].A_U), indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)]))
                if t > 0:
                    #terms.append(Term(c = net.gen[i].A_U, indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)] ))
                    terms.append(Term(c = round(-1 * net.gen[i].A_U), indices = [map_variable(i,min(k,len(net.gen[i].power_level[t-1])-1),t-1,len(net.gen[i].power_level[t]),net.time_period)] ))
                    #terms.append(Term(c = int(-1 * (net.gen[i].A_U + net.gen[i].A_D)),
                    #                  indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period),
                    #                            map_variable(i,min(k,len(net.gen[i].power_level[t-1])-1),t-1,len(net.gen[i].power_level[t]),net.time_period)]))
    ### Constraint
    ## Power demand
    for t in range(0, net.time_period):
        terms.append(Term(c = int(net.demand[t] * net.demand[t]), indices = [] ))
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                terms.append(Term(c = round(-2 * net.demand[t] * net.gen[i].power_level[t][k] ), indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)]))
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                terms.append(Term(c = round(net.gen[i].power_level[t][k]),indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period),
                                                    map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)]))
                for j in range(0, i):
                    for z in range(0, len(net.gen[j].power_level[t])):                
                        terms.append(Term(c = round(2 * net.gen[i].power_level[t][k] * net.gen[j].power_level[t][z]),
                                          indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period),
                                                    map_variable(j,z,t,len(net.gen[i].power_level[t]),net.time_period)]))
    ## Only one level can be on at the same time
    for t in range(0, net.time_period):
        for i in range(0, len(net.gen)):
            subterms = [] 
            for k in range(0, len(net.gen[i].power_level[t])):
                subterms.append(Term(c = 1, indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)]))
            subterms.append(Term(c=-1, indices=[]))
            terms.append(SlcTerm(subterms))
    ## Minimum Up-time Constraint
    for t in range(1, net.time_period):
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                subterms = []
                subterms.append(Term(c = 1 + net.gen[i].time_up_min, indices=[]))
                for z in range(t, min(t + net.gen[i].time_up_min, net.time_period)):
                    subterms.append(Term(c = -1,indices=[map_variable(i,k,z,len(net.gen[i].power_level[t]),net.time_period)] ))
                terms.append(SlcTerm(subterms))
    ## Minimum Down-Time Constraint
    for t in range(1, net.time_period):
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                subterms = []
                subterms.append(Term(c = 1 , indices=[]))
                for z in range(t, min(t + net.gen[i].time_down_min, net.time_period)):
                    subterms.append(Term(c = -1,indices=[map_variable(i,k,z,len(net.gen[i].power_level[t]),net.time_period)] ))
                terms.append(SlcTerm(subterms))
    ### Ramp rate up/down constraint
    for t in range(1, net.time_period):
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                for l in range(0, len(net.gen[i].power_level[t])):
                    subterms = []
                    subterms.append(Term(c = int(1 - net.gen[i].ramp_up), indices=[]))
                    subterms.append(Term(c = int(-1)* net.gen[i].power_level[t][l], indices=[map_variable(i,min(l,len(net.gen[i].power_level[t] )-1),t-1,len(net.gen[i].power_level[t]),net.time_period)]))                
                    subterms.append(Term(c = int(1)* net.gen[i].power_level[t][k], indices=[map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)]))
                    terms.append(SlcTerm(subterms))
    for t in range(1, net.time_period):
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                for l in range(0, len(net.gen[i].power_level[t])):
                    subterms = []
                    subterms.append(Term(c = int(1 - net.gen[i].ramp_down), indices=[]))
                    subterms.append(Term(c = int(1) * net.gen[i].power_level[t][l], indices=[map_variable(i,min(l,len(net.gen[i].power_level[t] )-1),t-1,len(net.gen[i].power_level[t]),net.time_period)]))                
                    subterms.append(Term(c = int(-1)* net.gen[i].power_level[t][k], indices=[map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)]))
                    terms.append(SlcTerm(subterms))
    #Spinning reserves constraint
    for t in range(1, net.time_period):
        subterms = []
        subterms.append(Term(c = int(net.demand[t] + net.reserves[t] + 1), indices=[]))
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                subterms.append(Term(c = int(-1 * net.gen[i].power_level[t][k]), indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)] ))
        terms.append(SlcTerm(subterms))
    # Must run units
    for t in range(1, net.time_period):
        subterms = []
        subterms.append(Term(c = MRU, indices=[]))
        for i in range(0, len(net.gen)):
            for k in range(0, len(net.gen[i].power_level[t])):
                subterms.append(Term(c = int(-1), indices = [map_variable(i,k,t,len(net.gen[i].power_level[t]),net.time_period)] ))
        terms.append(SlcTerm(subterms))
    return Problem(name="Unit Commitment", problem_type=ProblemType.pubo, terms=terms)

In [8]:
solver = PopulationAnnealing(workspace)  
f_test = open ('json_file_path', "r")
# Reading from file
data_test = json.loads(f_test.read())
net_test = create_network(data_test, time = 12)
net_test.get_all_generators()
print('Submitting problem...')
OptimizationProblem_to = UCproblem(net_test)
job_to = solver.submit(OptimizationProblem_to)
print(job_to.id)

Submitting problem...
1b6f2aee-1ce3-11ed-8ad6-e6aaeac94a19
